<a href="https://colab.research.google.com/github/alexsantostavares/ArvoreDeDecisao/blob/master/ArvoreDecisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#importando bibliotecas
import pandas as pd
import numpy as np

#
from sklearn.tree import DecisionTreeClassifier as dtc # tree algorithm
from sklearn.model_selection import train_test_split # splitting the data
from sklearn.metrics import accuracy_score # model precision
from sklearn.tree import plot_tree # tree diagram

In [13]:
# abrindo arquivo csv
base = pd.read_csv('/content/drive/MyDrive/Base/BD-2trab(CSV) (1).csv')
base

,Alternativo,Bar,Sex/Sab,Faminto,Cliente,Preço,Chove,Reserva,Tipo,Espera,Meta
0,Sim,Não,Não,Sim,Alg,$$$,Não,Sim,Fra,0-10,Sim
1,Sim,Não,Não,Sim,Che,$,Não,Não,Tai,30-60,Não
2,Não,Sim,Não,Não,Alg,$,Não,Não,Ham,0-10,Sim
3,Sim,Não,Sim,Sim,Che,$,Sim,Não,Tai,10-30,Sim
4,Sim,Não,Sim,Não,Che,$$$,Não,Sim,Fra,>60,Não
5,Não,Sim,Não,Sim,Alg,$$,Sim,Sim,Ita,0-10,Sim
6,Não,Sim,Não,Não,Ne,$,Sim,Não,Ham,0-10,Não
7,Não,Não,Não,Sim,Alg,$$,Sim,Sim,Tai,0-10,Sim
8,Não,Sim,Sim,Não,Che,$,Sim,Não,Ham,>60,Não
9,Sim,Sim,Sim,Sim,Che,$$$,Não,Sim,Ita,10-30,Não


In [14]:
# substituindo sim e não por 1 e 0
base = base.replace(['Sim'], 1)
base = base.replace(['Sim '], 1)
base = base.replace([' Sim'], 1)
base = base.replace(['sim'], 1)
base = base.replace(['sim '], 1)
base = base.replace([' sim'], 1)
base = base.replace(['Não'], 0)
base = base.replace(['Não '], 0)
base = base.replace([' Não'], 0)
base = base.replace(['não'], 0)
base = base.replace(['não '], 0)
base = base.replace([' não'], 0)

# converte string repetidas em valores numericos

mapeamento_pala = {}

for coluna in base.columns:
  if type(coluna) is str:
    contador = base[coluna].unique()
    total = len(contador)
    mapeando = dict(zip(contador, range(total)))
    mapeamento_pala[coluna] = mapeando
    base[coluna] = base[coluna].replace(mapeando)

base

,Alternativo,Bar,Sex/Sab,Faminto,Cliente,Preço,Chove,Reserva,Tipo,Espera,Meta
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,1,1,1,1
2,1,1,0,1,0,1,0,1,2,0,0
3,0,0,1,0,1,1,1,1,1,2,0
4,0,0,1,1,1,0,0,0,0,3,1
5,1,1,0,0,0,2,1,0,3,0,0
6,1,1,0,1,2,1,1,1,2,0,1
7,1,0,0,0,0,2,1,0,1,0,0
8,1,1,1,1,1,1,1,1,2,3,1
9,0,1,1,0,1,0,0,0,3,2,1


In [15]:

# calcula a entropia
def calculo_de_entropia(df_label):
  classes, class_counts = np.unique(df_label, return_counts = True)
  valor_de_entropia = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) for i in range(len(classes))])
  return valor_de_entropia

# recebe a entropia calculada e calcula o ganho de informação
def ganho_de_informacao(dataset, feature, label):
  entropia_dataset = calculo_de_entropia(dataset[label])
  valores, feat_counts = np.unique(dataset[feature], return_counts=True)

  entropia_pesada = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculo_de_entropia(dataset.where(dataset[feature] == valores[i]).dropna()[label])for i in range(len(valores))])

  ganho_de_informacao = entropia_dataset - entropia_pesada
  return ganho_de_informacao

# monta a arvore de decisao
def arvore_de_decisao(dataset, base, features, label, parente):
  numdata = np.unique(base[label], return_counts=True)
  dado_unico = np.unique(dataset[label])
  
  if len(dado_unico) <= 1:
    return dado_unico[0]
  elif len(dataset) == 0:
    return dado_unico[np.argmax(numdata)]
  elif len(features) == 0:
    return parente
  else:
    parente = dado_unico[np.argmax(numdata[1])]

    valores_do_item = [ganho_de_informacao(dataset, feature, label) for feature in features]

    index_otimizado = np.argmax(valores_do_item)
    feature_otimizado = features[index_otimizado]
    arvoreDeDecisao = {feature_otimizado:{}}
    features = [i for i in features if i != feature_otimizado]

    for valor in np.unique(dataset[feature_otimizado]):
      min_data = dataset.where(dataset[feature_otimizado] == valor).dropna()

      arvore_minima = arvore_de_decisao(min_data, base, features, label, parente)

      arvoreDeDecisao[feature_otimizado][valor] = arvore_minima

    return(arvoreDeDecisao)

# a partir da arvore verifica a previsão da meta
def previsao_meta(dadosDeTeste, arvoreDeDecisao):
  for nodes in arvoreDeDecisao.keys():
    valor = dadosDeTeste[nodes]
    arvoreDeDecisao = arvoreDeDecisao[nodes][valor]

    previsao = 0

    if type(arvoreDeDecisao) is dict:

      previsao = previsao_meta(dadosDeTeste, arvoreDeDecisao)
    else:
      previsao = arvoreDeDecisao
      break;
  return previsao



In [75]:

# definindo as colunas usadas na formação da arvore e no calculo de previsao
features = base.columns[:-1]
label = 'Meta'
parent = None
features

features2 = base[base.columns[:-1]].values
label2 = base[base.columns[-1]].values

# Mostrando a eficiencia
X_train, X_test, y_train, y_test = train_test_split(features2, label2, test_size = 0.2, random_state = 0)
model = dtc(criterion = 'entropy', max_depth = 5)
model.fit(X_train, y_train)

pred_model = model.predict(X_test)
pred_model2 = model.predict(features2)

In [20]:
# chama a funcao que monta a arvore com a base de dados e o que definimos antes de parametros
arvoreDeDecisao = arvore_de_decisao(base, base, features, label, parent)

In [21]:
# testando a precisao com uma entrada qualquer
dados_de_teste = {'Alternativo': 1, 
                  'Bar': 1, 
                  'Sex/Sab': 0, 
                  'Faminto': 1, 
                  'Cliente': 1, 
                  'Preço': 2, 
                  'Chove': 1, 
                  'Reserva': 1, 
                  'Tipo': 3, 
                  'Espera': 0}
teste_de_dados = pd.Series(dados_de_teste)
previsao = previsao_meta(teste_de_dados, arvoreDeDecisao)
int(previsao)

1

In [76]:

print('precisão teste {:.0%}'.format(accuracy_score(y_test, pred_model), attrs = ['bold']))

print('precisão total {:.0%}'.format(accuracy_score(label2, pred_model2), attrs = ['bold']))

precisão teste 67%
precisão total 92%
